In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import warnings
warnings.filterwarnings("ignore")

## read

`openai/clip-vit-base-patch32`

In [ ]:
PATH_METADATA = "../results"
MODEL_NAME = "openai/clip-vit-base-patch32"

In [ ]:
results = pd.concat([
    pd.read_csv(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_fixlip.csv")),
    pd.read_csv(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_game_gradeclip.csv")),
    pd.read_csv(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_exclip.csv"))
])

results_details = np.load(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_fixlip.npy"), allow_pickle=True).item() |\
    np.load(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_game_gradeclip.npy"), allow_pickle=True).item() |\
    np.load(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_exclip.npy"), allow_pickle=True).item()

or `openai/clip-vit-base-patch16`

In [ ]:
MODEL_NAME = "openai/clip-vit-base-patch16"

In [ ]:
results = pd.concat([pd.read_csv(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_game_gradeclip.csv"))] +\
                    [pd.read_csv(os.path.join(PATH_METADATA, MODEL_NAME, f'mscoco_aid_fixlip_{start}_{start+100}.csv')) for start in list(range(0, 1000, 100))])

results_details = np.load(os.path.join(PATH_METADATA, MODEL_NAME, "mscoco_aid_game_gradeclip.npy"), allow_pickle=True).item() |\
      np.load(os.path.join(PATH_METADATA, MODEL_NAME, f'mscoco_aid_fixlip_0_100.npy'), allow_pickle=True).item()
for start in list(range(100, 1000, 100)):
    temp = np.load(os.path.join(PATH_METADATA, MODEL_NAME, f'mscoco_aid_fixlip_{start}_{start+100}.npy'), allow_pickle=True).item()
    for k, v in temp.items():
        results_details[k] = results_details[k] | temp[k]

or `google/siglip2-base-patch32-256`

In [ ]:
PATH_METADATA = "../results/mscoco/google/siglip2-base-patch32-256"
MODEL_NAME = "google/siglip2-base-patch32-256"

In [ ]:
results_details = {}
results_details['shapley'] = np.load(os.path.join(PATH_METADATA, str(2**17), "shapley", "mscoco_aid_fixlip.npy"), allow_pickle=True).item()
mode = "banzhaf"
for p_sampler in [0.3, 0.4, 0.5, 0.6, 0.7]:
    results_details[f'{mode}/{p_sampler}'] = np.load(os.path.join(PATH_METADATA, str(2**17), mode, str(p_sampler), 'mscoco_aid_fixlip.npy'), allow_pickle=True).item()
mode = "fixlip"
for p_sampler in [0.3, 0.4, 0.5, 0.6, 0.7]:
    results_details[f'{mode}/{p_sampler}'] = np.load(os.path.join(PATH_METADATA, str(2**21), mode, str(p_sampler), 'mscoco_aid_fixlip.npy'), allow_pickle=True).item()

## preprocess

In [ ]:
grid_points = np.linspace(0.00, 1.00, 51)

#### clip

In [ ]:
results_simplified = pd.DataFrame({
    'input': [], 
    'method': [],
    'mode': [], 
    'order': [], 
    'curve': [],
    'x': [],
    'y': []
})
areas = []

In [ ]:
for mode in [
    'banzhaf/0.7', 'banzhaf/0.5', 'banzhaf/0.3', 'shapley', 
    'gradeclip', 'game', 
    'exclip'
]:
    for order in [1, 2]:
        if mode in ['game', 'gradeclip']:
            if order == 2:
                continue
            method = mode
        elif mode in ['exclip']:
            if order == 1:
                continue
            method = mode
        else:
            method = f'{mode}/order{order}'
        for k, v in results_details[method].items():
            predictions_deletion_mif = v['predictions_deletion_mif']
            predictions_deletion_lif = v['predictions_deletion_lif']

            assert predictions_deletion_mif[-1] == predictions_deletion_lif[-1]
            assert predictions_deletion_mif[0] == predictions_deletion_lif[0]
            
            # normalize the curve
            min_value = predictions_deletion_mif[-1]
            max_value = predictions_deletion_mif[0]

            predictions_deletion_mif_01 = (predictions_deletion_mif - min_value) / (max_value - min_value)
            predictions_deletion_lif_01 = (predictions_deletion_lif - min_value) / (max_value - min_value)
            
            xvals = np.arange(len(predictions_deletion_mif)) / (len(predictions_deletion_mif) - 1)
            y_mif = np.interp(grid_points, xvals, predictions_deletion_mif_01)
            y_lif = np.interp(grid_points, xvals, predictions_deletion_lif_01)

            results_simplified = pd.concat([results_simplified, pd.DataFrame({
                'input': k, 
                'method': method,
                'mode': mode, 
                'order': order,
                'curve': 'mif',
                'x': grid_points,
                'y': y_mif
            }), pd.DataFrame({
                'input': k, 
                'method': method,
                'mode': mode, 
                'order': order,
                'curve': 'lif',
                'x': grid_points,
                'y': y_lif
            })])

            area_mif = np.trapezoid(y_mif, x=grid_points)
            area_lif = np.trapezoid(y_lif, x=grid_points)
            area = area_lif - area_mif

            areas.append({
                'input': k,
                'method': method,
                'mode': mode, 
                'order': order,
                'area': area
            })

In [ ]:
results_simplified.shape

In [ ]:
results_aggregated = pd.DataFrame(areas).groupby(['mode', 'order']).aggregate({'area': ['mean', 'std']})
results_aggregated

#### siglip

In [ ]:
results_simplified = pd.DataFrame({
    'input': [], 
    'method': [],
    'mode': [], 
    'curve': [],
    'x': [],
    'y': []
})
areas = []

methods = ['shapley']
for mode in ["fixlip", "banzhaf"]:
    for p_sampler in [0.7, 0.6, 0.5, 0.4, 0.3]:
        methods += [f'{mode}/{p_sampler}']

In [ ]:
for method in methods:
    for k, v in results_details[method].items():
        predictions_deletion_mif = v['predictions_deletion_mif']
        predictions_deletion_lif = v['predictions_deletion_lif']

        assert predictions_deletion_mif[-1] == predictions_deletion_lif[-1]
        assert predictions_deletion_mif[0] == predictions_deletion_lif[0]
        
        # normalize the curve
        min_value = predictions_deletion_mif[-1]
        max_value = predictions_deletion_mif[0]

        predictions_deletion_mif_01 = (predictions_deletion_mif - min_value) / (max_value - min_value)
        predictions_deletion_lif_01 = (predictions_deletion_lif - min_value) / (max_value - min_value)
        
        xvals = np.arange(len(predictions_deletion_mif)) / (len(predictions_deletion_mif) - 1)
        y_mif = np.interp(grid_points, xvals, predictions_deletion_mif_01)
        y_lif = np.interp(grid_points, xvals, predictions_deletion_lif_01)

        results_simplified = pd.concat([results_simplified, pd.DataFrame({
            'input': k, 
            'method': method,
            'curve': 'mif',
            'x': grid_points,
            'y': y_mif
        }), pd.DataFrame({
            'input': k, 
            'method': method,
            'curve': 'lif',
            'x': grid_points,
            'y': y_lif
        })])

        area_mif = np.trapezoid(y_mif, x=grid_points)
        area_lif = np.trapezoid(y_lif, x=grid_points)
        area = area_lif - area_mif

        areas.append({
            'input': k,
            'method': method,
            'area': area
        })

In [ ]:
results_simplified.shape

In [ ]:
results_aggregated = pd.DataFrame(areas).groupby(['method']).aggregate({'area': ['mean', 'std']})
results_aggregated

## visualize

In [ ]:
SMALL_SIZE = 11
MEDIUM_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize

#### clip (ViT-B/32)

In [ ]:
df_plot = results_simplified.loc[results_simplified.method.isin([
    'shapley/order1', 
    'banzhaf/0.3/order2', 
    'banzhaf/0.5/order2', 
    'banzhaf/0.7/order2', 
    'game', 'gradeclip', 'exclip'
]),]
df_plot.curve = df_plot.curve.replace({'mif': 'delete important (lower better)', 'lif': 'insert important (higher better)'})
df_plot.method = df_plot.method.replace({
    'shapley/order1': f'Shapley values (${results_aggregated.loc[("shapley", 1),:][0].item():.02f} \pm {results_aggregated.loc[("shapley", 1),:][1].item():.02f}$)', 
    'banzhaf/0.3/order2': f'FIxLIP $p=0.3$ (${results_aggregated.loc[("banzhaf/0.3", 2),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.3", 2),:][1].item():.02f}$)', 
    'banzhaf/0.5/order2': f'FIxLIP $p=0.5$ (${results_aggregated.loc[("banzhaf/0.5", 2),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.5", 2),:][1].item():.02f}$)', 
    'banzhaf/0.7/order2': f'FIxLIP $p=0.7$ (${results_aggregated.loc[("banzhaf/0.7", 2),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.7", 2),:][1].item():.02f}$)', 
    'gradeclip': f'Grad-ECLIP  (${results_aggregated.loc[("gradeclip", 1),:][0].item():.02f} \pm {results_aggregated.loc[("gradeclip", 1),:][1].item():.02f}$)', 
    'game': f'GAME (${results_aggregated.loc[("game", 1),:][0].item():.02f} \pm {results_aggregated.loc[("game", 1),:][1].item():.02f}$)', 
    'exclip': f'exCLIP (${results_aggregated.loc[("exclip", 2),:][0].item():.02f} \pm {results_aggregated.loc[("exclip", 2),:][1].item():.02f}$)', 
})

In [ ]:
plt.figure(figsize=(8, 3))
ax = sns.lineplot(
    data=df_plot, 
    x='x', y='y', 
    hue='method', palette=[
        '#EA4335', #'#EA4335', 
        '#E37400', #'#E37400', 
        '#FBBC04', #'#FBBC04', 
        '#4285F4', #'#4285F4', 
        '#34A853', '#9AA0A6', "#CC8899"
    ],
    style='curve', dashes=[(4, 1), (1, 1)], style_order=["insert important (higher better)", "delete important (lower better)"],
    errorbar="se",
    linewidth=2
)
handles, labels = ax.get_legend_handles_labels()
labels[0] = "$\mathbf{Method}$ ($\mathbf{Area}$ $\mathbf{between}$ $\mathbf{curves}$)"
labels[8] = "$\mathbf{Curve}$"
ax.legend(handles, labels)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1.035))
plt.xlabel("Percentage of input $k\;/\;(n_{\mathrm{txt}} + n_{\mathrm{img}})$")
plt.xlim([0, 1.0])
ax.set_xticklabels(["100%", "80%", "60%", "40%", "20%", "0%"])
xticks = ax.xaxis.get_majorticklabels()
xticks[0].set_horizontalalignment('left')
xticks[-1].set_horizontalalignment('right')
plt.ylabel("Prediction change (normalized)")
plt.ylim([-0.6, 1.1])
# yticks = ax.yaxis.get_majorticklabels()
# yticks[0].set_verticalalignment('bottom')
# yticks[-1].set_verticalalignment('top')
plt.tight_layout(pad=0.05)
ax.yaxis.set_label_coords(-0.15, 0.47)
plt.savefig(f'insertion_deletion_{MODEL_NAME.replace("/", "-")}.pdf')

#### clip (ViT-B/16)

In [ ]:
df_plot = results_simplified.loc[results_simplified.method.isin([
    'banzhaf/0.3/order2', 
    'banzhaf/0.5/order1', 
    'banzhaf/0.5/order2', 
    'banzhaf/0.7/order2', 
    'game', 'gradeclip'
]),]
df_plot.curve = df_plot.curve.replace({'mif': 'delete important (lower better)', 'lif': 'insert important (higher better)'})
df_plot.method = df_plot.method.replace({
    'banzhaf/0.3/order2': f'FIxLIP $p=0.3$ (${results_aggregated.loc[("banzhaf/0.3", 2),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.3", 2),:][1].item():.02f}$)', 
    'banzhaf/0.5/order1': f'Banzhaf values (${results_aggregated.loc[("banzhaf/0.5", 1),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.5", 1),:][1].item():.02f}$)', 
    'banzhaf/0.5/order2': f'FIxLIP $p=0.5$ (${results_aggregated.loc[("banzhaf/0.5", 2),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.5", 2),:][1].item():.02f}$)', 
    'banzhaf/0.7/order2': f'FIxLIP $p=0.7$ (${results_aggregated.loc[("banzhaf/0.7", 2),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.7", 2),:][1].item():.02f}$)', 
    'gradeclip': f'Grad-ECLIP  (${results_aggregated.loc[("gradeclip", 1),:][0].item():.02f} \pm {results_aggregated.loc[("gradeclip", 1),:][1].item():.02f}$)', 
    'game': f'GAME (${results_aggregated.loc[("game", 1),:][0].item():.02f} \pm {results_aggregated.loc[("game", 1),:][1].item():.02f}$)'
})

In [ ]:
plt.figure(figsize=(8, 3))
ax = sns.lineplot(
    data=df_plot, 
    x='x', y='y', 
    hue='method', palette=[
        '#EA4335', #'#EA4335', 
        '#E37400', #'#E37400', 
        '#FBBC04', #'#FBBC04', 
        '#4285F4', # '#4285F4', 
        '#34A853', '#9AA0A6'
    ], 
    hue_order=df_plot.method.unique()[[0, 2, 3, 1, 4, 5]],
    style='curve', dashes=[(4, 1), (1, 1)], style_order=["insert important (higher better)", "delete important (lower better)"],
    errorbar="se",
    linewidth=2
)
handles, labels = ax.get_legend_handles_labels()
labels[0] = "$\mathbf{Method}$ ($\mathbf{Area}$ $\mathbf{between}$ $\mathbf{curves}$)"
labels[7] = "$\mathbf{Curve}$"
ax.legend(handles, labels)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1.035))
plt.xlabel("Percentage of input $k\;/\;(n_{\mathrm{txt}} + n_{\mathrm{img}})$")
plt.xlim([0, 1.0])
ax.set_xticklabels(["100%", "80%", "60%", "40%", "20%", "0%"])
xticks = ax.xaxis.get_majorticklabels()
xticks[0].set_horizontalalignment('left')
xticks[-1].set_horizontalalignment('right')
plt.ylabel("Prediction change (normalized)")
plt.ylim([-0.3, 1.1])
def ascii_minus(x, _):
    return f"{x:.2f}".replace('-', '\u2212')
ax.yaxis.set_major_formatter(FuncFormatter(ascii_minus))
# yticks = ax.yaxis.get_majorticklabels()
# yticks[0].set_verticalalignment('bottom')
# yticks[-1].set_verticalalignment('top')
plt.tight_layout(pad=0.05)
ax.yaxis.set_label_coords(-0.15, 0.47)
plt.savefig(f'insertion_deletion_{MODEL_NAME.replace("/", "-")}.pdf')

#### siglip

In [ ]:
df_plot = results_simplified.loc[results_simplified.method.isin([
    'shapley', 
    "banzhaf/0.3",
    "banzhaf/0.4",
    "banzhaf/0.5",
    "banzhaf/0.6",
    "banzhaf/0.7",
]),]
df_plot.curve = df_plot.curve.replace({'mif': 'delete important (lower better)', 'lif': 'insert important (higher better)'})
df_plot.method = df_plot.method.replace({
    'shapley': f'FIxLIP SI (${results_aggregated.loc[("shapley"),:][0].item():.02f} \pm {results_aggregated.loc[("shapley"),:][1].item():.02f}$)', 
    'banzhaf/0.3': f'FIxLIP $p=0.3$ (${results_aggregated.loc[("banzhaf/0.3"),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.3"),:][1].item():.02f}$)', 
    'banzhaf/0.4': f'FIxLIP $p=0.4$ (${results_aggregated.loc[("banzhaf/0.4"),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.4"),:][1].item():.02f}$)', 
    'banzhaf/0.5': f'FIxLIP $p=0.5$ (${results_aggregated.loc[("banzhaf/0.5"),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.5"),:][1].item():.02f}$)', 
    'banzhaf/0.6': f'FIxLIP $p=0.6$ (${results_aggregated.loc[("banzhaf/0.6"),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.6"),:][1].item():.02f}$)', 
    'banzhaf/0.7': f'FIxLIP $p=0.7$ (${results_aggregated.loc[("banzhaf/0.7"),:][0].item():.02f} \pm {results_aggregated.loc[("banzhaf/0.7"),:][1].item():.02f}$)', 
})

In [ ]:
plt.figure(figsize=(8, 3))
ax = sns.lineplot(
    data=df_plot, 
    x='x', y='y', 
    hue='method', palette=[
        '#EA4335', #'#EA4335',
        '#E75C1B', 
        '#E37400', #'#E37400', 
        '#EF9802',
        '#FBBC04', #'#FBBC04', 
        '#4285F4', #'#4285F4', 
    ],
    style='curve', dashes=[(4, 1), (1, 1)], style_order=["insert important (higher better)", "delete important (lower better)"],
    errorbar=("se", 0.25),
    linewidth=2
)
handles, labels = ax.get_legend_handles_labels()
labels[0] = "$\mathbf{Method}$ ($\mathbf{Area}$ $\mathbf{between}$ $\mathbf{curves}$)"
labels[7] = "$\mathbf{Curve}$"
ax.legend(handles, labels)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1.035))
plt.xlabel("Percentage of input $k\;/\;(n_{\mathrm{txt}} + n_{\mathrm{img}})$")
plt.xlim([0, 1.0])
ax.set_xticklabels(["100%", "80%", "60%", "40%", "20%", "0%"])
xticks = ax.xaxis.get_majorticklabels()
xticks[0].set_horizontalalignment('left')
xticks[-1].set_horizontalalignment('right')
plt.ylabel("Prediction change (normalized)")
plt.ylim([-0.50, 1.25])
yticks = ax.yaxis.get_majorticklabels()
# yticks[0].set_verticalalignment('bottom')
yticks[-1].set_verticalalignment('top')
plt.tight_layout(pad=0.05)
ax.yaxis.set_label_coords(-0.15, 0.47)
plt.savefig(f'insertion_deletion_{MODEL_NAME.replace("/", "-")}.pdf')

## insertion/deletion example

In [ ]:
df_plot = results_simplified.loc[results_simplified.method.isin([
    'shapley/order2', 
    'gradeclip', 
]),]
df_plot.curve = df_plot.curve.replace({'mif': 'delete important (↓)', 'lif': 'insert important (↑)'})
df_plot.method = df_plot.method.replace({
    'shapley/order2': f'second-order interactions', 
    'gradeclip': f'first-order attributions', 
})

In [ ]:
plt.figure(figsize=(7, 3))
ax = sns.lineplot(
    data=df_plot.loc[df_plot.input == 2892, :], 
    x='x', y='y', 
    hue='method', palette=[ 
        '#EA4335',
        '#34A853',
    ],
    style='curve', dashes=[(4, 1), (1, 1)], 
    style_order=["insert important (↑)", "delete important (↓)"],
    errorbar="se",
    linewidth=2
)
handles, labels = ax.get_legend_handles_labels()
labels[0] = "$\mathbf{Explanation}$ $\mathbf{method}$"
labels[3] = "$\mathbf{Evaluation}$ $\mathbf{curve}$"
ax.legend(handles, labels)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1.035))
plt.xlabel("Percentage of input $k\;/\;(n_{\mathrm{txt}} + n_{\mathrm{img}})$")
plt.xlim([0, 1.0])
ax.set_xticklabels(["100%", "80%", "60%", "40%", "20%", "0%"])
xticks = ax.xaxis.get_majorticklabels()
xticks[0].set_horizontalalignment('left')
xticks[-1].set_horizontalalignment('right')
plt.ylabel("Prediction change (normalized)")
plt.ylim([-0.8, 1.3])
plt.vlines(1 - np.array([3, 9, 25, 45, 57]) / 72, [-0.81]*5, [1.31]*5, colors='lightgray', linestyles='solid', label=None)
# yticks = ax.yaxis.get_majorticklabels()
# yticks[0].set_verticalalignment('bottom')
# yticks[-1].set_verticalalignment('top')
plt.tight_layout(pad=0.05)
ax.yaxis.set_label_coords(-0.15, 0.47)
plt.savefig(f'appendix_insertion_deletion_curve.pdf')